In [1]:
from jl_song_data import SongData 
import pandas as pd 
from jl_chord_cleaning import ChordDatasetCleaner

In [2]:
data = SongData('../data/chords_clean_1_2_3_4.csv')

In [3]:
df = data.df

In [4]:
df

,url,name,decade,genre,chords,uuid
0,https://tabs.ultimate-guitar.com/tab/10000_man...,Whats The Matter Here,1980s,Folk,"['G', 'G', 'C', 'D', 'G', 'G', 'G', 'G', 'G', ...",828ba8e4-d791-4403-a4cc-5ca4a1c0cc1a
1,https://tabs.ultimate-guitar.com/tab/1002296,Limón Y Sal (ver 2),2000s,Pop,"['F', 'G', 'Am', 'G', 'C', 'F', 'F', 'G', 'Am'...",bd5b9396-8d6a-49ee-a738-6577cf310783
2,https://tabs.ultimate-guitar.com/tab/1055161,Time To Say Goodbye Con Te Partirò,1990s,Pop%%Classical%%Pop,"['G', 'D', 'Em', 'C', 'G', 'D', 'Em', 'C', 'G'...",d42333bf-e925-4ad1-a4b9-bfe2e2df209e
3,https://tabs.ultimate-guitar.com/tab/1060259,El Mañana (ver 3),2000s,Electronic%%Pop,"['Am', 'Em/G', 'F', 'Em', 'Bm', 'Dm', 'Am', 'E...",d6123fcf-d023-4425-8644-0a04afd88ebf
4,https://tabs.ultimate-guitar.com/tab/1061244,Balladen Om Herr Fredrik Åkare Och Den Söta Fr...,1960s,Jazz,"['Dm', 'E', 'Am', 'Dm', 'G7', 'C', 'E7', 'Am',...",8bc3d690-78cd-4655-add3-062e7f9df1a7
...,...,...,...,...,...,...
10672,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,Lighthouse,2010s,Reggae,"['C', 'F', 'C', 'G', 'C', 'C', 'F', 'C', 'G', ...",fda4b7b8-e3bb-4c38-b5a7-73762c8c8261
10673,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,Personal Revolution,2010s,Reggae,"['Bm', 'A', 'Bm', 'A', 'Bm', 'A', 'Em', 'G', '...",089f0438-4e4d-44dd-83e5-5dd929753f0f
10674,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,True To Myself,2000s,Reggae%%Reggae,"['A', 'E', 'Bm', 'D', 'A', 'E', 'Bm', 'D', 'A'...",c0d75c52-ae53-4b47-bc8e-a723ba542d00
10675,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,True To Myself (ver 2),2000s,Reggae,"['A', 'E', 'Bm', 'A', 'E', 'Bm', 'A', 'E', 'Bm...",29047e58-5391-4875-ab94-3a5a7fdbcd0e


In [5]:
df['chords'] = df['chords'].apply(eval)

In [6]:
def get_all_chords(dataframe):
    all_chords = []

    for chord_list in dataframe['chords']:
        all_chords.extend(chord_list)
    
    return all_chords

In [7]:
all_chords = get_all_chords(df)
len(all_chords)

923833

In [8]:
def get_unique_chords(dataframe):
    unique_chords = set([])

    for chord_list in dataframe['chords']:
        unique_chords = unique_chords.union(set(chord_list))

    unique_chords = list(unique_chords)  
    return unique_chords

In [9]:
unique_chords = get_unique_chords(df)
len(unique_chords)

1976

In [10]:
from pychord_jl import Chord
def replace_german_system(input):
    if input[0] == 'H':
        return 'Bb'+input[1:]
    
    return input

def replace_6_9(input):
    return (input
            .replace('6/9', '6_9')
            .replace('7/9', '7_9')
            .replace('7/5', '7_5')
            .replace('7/6', '7_6')
           )

def replace_enharmonic(input):
    return (input
            .replace('E#', 'F')
            .replace('B#', 'C')
            .replace('Fb', 'E')
            .replace('Cb', 'B')
           )

def replace_errors(input):
    return (input
            .replace('G(2)', 'G')
            .replace('A*', 'A')
            .replace('(III)', '')
            .replace('Gm#', 'G#m')
           )

def parse(input):
    input = replace_german_system(input)
    input = replace_6_9(input)
    input = replace_enharmonic(input)
    input = replace_errors(input)
    
    try:
        return Chord(input)
    except:
        return None

    
    

In [11]:
black_list = ['Go', 'AM']

In [12]:
parsed_chords = [ (parse(chord_str),chord_str) for chord_str in unique_chords if chord_str not in black_list]

In [13]:
def count_appearances(chord_str):
    return len([chord for chord in all_chords if chord == chord_str])

In [14]:
not_found = [ (count_appearances(x[1]),x[1]) for x in parsed_chords if x[0] is None]
len(not_found)

212

In [15]:
import functools 
total_not_found = functools.reduce(lambda x,y:x+y, [ count_appearances(x[1]) for x in parsed_chords if x[0] is None])

In [16]:
not_found

[(2, 'B7dim7/D#'),
 (3, 'Cmin6'),
 (2, 'F#m7+5'),
 (4, 'B-'),
 (5, 'G/H'),
 (3, 'F#6 (add9)'),
 (2, 'B6add11'),
 (1, 'E(dim7)'),
 (1, 'Fmaj7#5'),
 (1, 'F(6)'),
 (1, 'Ab(IV)'),
 (1, 'D(4)'),
 (2, 'E-9'),
 (1, 'CMin'),
 (3, 'F13(b9)'),
 (20, 'G(b5)'),
 (2, 'Bm7/9'),
 (2, 'Am7+'),
 (1, 'Fdim3'),
 (5, 'Em+'),
 (1, 'D(V)'),
 (1, 'Gb7(b5)'),
 (1, 'B5b'),
 (2, 'C(maj7)'),
 (1, 'DmMaj7'),
 (1, 'Fma9/G'),
 (3, 'F#7b9sus4'),
 (5, 'F(V)'),
 (35, 'G6add11'),
 (1, 'F#(2)/A#'),
 (5, 'D#(b5)'),
 (4, 'GMaj'),
 (3, 'C3'),
 (1, 'Bb7(b13)'),
 (6, 'G(maj7)'),
 (3, 'Cm7/9'),
 (2, 'AMaj'),
 (2, 'F#Min'),
 (5, 'C7#5#9'),
 (1, 'Dm7+5'),
 (1, 'Bb7(b5)'),
 (3, 'E7*'),
 (1, 'Bsus4dim7'),
 (4, 'D(b5)'),
 (1, 'LA4'),
 (3, 'Cm*'),
 (1, 'EDim'),
 (1, 'Esus4(add13)'),
 (6, 'B7/13'),
 (2, 'G#addb13'),
 (2, 'F#m7#5'),
 (10, 'D7#'),
 (2, 'E7(b13)'),
 (18, 'C*'),
 (2, 'B7*'),
 (1, 'A(maj)'),
 (1, 'G(m)'),
 (7, 'E7/9b'),
 (5, 'C#9(#5)'),
 (1, 'G7(b13)'),
 (2, 'RE'),
 (1, 'DMaj7]'),
 (3, 'DMin'),
 (3, 'Bbj7'),
 (1, 'Cm-5')

In [17]:
not_recognised_chords = [chord[1] for chord in not_found]
not_recognised_chords

['B7dim7/D#',
 'Cmin6',
 'F#m7+5',
 'B-',
 'G/H',
 'F#6 (add9)',
 'B6add11',
 'E(dim7)',
 'Fmaj7#5',
 'F(6)',
 'Ab(IV)',
 'D(4)',
 'E-9',
 'CMin',
 'F13(b9)',
 'G(b5)',
 'Bm7/9',
 'Am7+',
 'Fdim3',
 'Em+',
 'D(V)',
 'Gb7(b5)',
 'B5b',
 'C(maj7)',
 'DmMaj7',
 'Fma9/G',
 'F#7b9sus4',
 'F(V)',
 'G6add11',
 'F#(2)/A#',
 'D#(b5)',
 'GMaj',
 'C3',
 'Bb7(b13)',
 'G(maj7)',
 'Cm7/9',
 'AMaj',
 'F#Min',
 'C7#5#9',
 'Dm7+5',
 'Bb7(b5)',
 'E7*',
 'Bsus4dim7',
 'D(b5)',
 'LA4',
 'Cm*',
 'EDim',
 'Esus4(add13)',
 'B7/13',
 'G#addb13',
 'F#m7#5',
 'D7#',
 'E7(b13)',
 'C*',
 'B7*',
 'A(maj)',
 'G(m)',
 'E7/9b',
 'C#9(#5)',
 'G7(b13)',
 'RE',
 'DMaj7]',
 'DMin',
 'Bbj7',
 'Cm-5',
 'D(II)',
 'Ebmaj7/Em7b5',
 'Gmaj7+9',
 'Bbmaj9+11',
 'LA7',
 'G#(IV)',
 'Dadd4add9',
 'F#m7(b5)',
 'Ab13(b9)',
 'Ab(maj9)',
 'Em6add9',
 'C#mb5',
 'C#m*',
 'Gm7b5b9',
 'C6add4',
 'Eb(maj7)',
 'Fmaj7-5',
 'Bbmaj7+5',
 'Eb(dim7)',
 'B7(b5)',
 'B11b9',
 'Am-5',
 'N.C.',
 'A(maj7)',
 'C+9',
 'B7+5+9',
 'G3',
 'A6sus4',
 'Badd#11

In [18]:
def contains_not_recognised_chord(chords):
    return any([not_recognised in chords for not_recognised in not_recognised_chords])

In [25]:
print(f'{100 * len(df[df["chords"].apply(contains_not_recognised_chord)]) / len(df)}% of songs with no recognised chords')

1.6577690362461366% of songs with no recognised chords


In [21]:
print(f'{100 * total_not_found / len(all_chords)}% of chords not recognised')

0.08670398221323551% of chords not recognised


In [36]:
not_found.sort(key = lambda x: x[0], reverse = True) 

In [37]:
not_found

[(35, 'G6add11'),
 (21, 'Em6add9'),
 (18, 'C*'),
 (17, 'Fadd#11'),
 (16, 'CaddG'),
 (15, 'BB'),
 (14, 'G(b5)'),
 (12, 'Emmaj9'),
 (11, 'F(b5)'),
 (9, 'Cma9'),
 (9, 'Dadd4add9'),
 (8, 'C#b7b9'),
 (7, 'F+7'),
 (7, '(E)'),
 (7, 'C(2)'),
 (7, 'E7/9b'),
 (7, 'Dm(add9)/A'),
 (6, 'Am/Fnat'),
 (6, 'F#7add4'),
 (6, 'B7/13'),
 (6, 'B7/13b'),
 (6, 'D-'),
 (6, 'C(7b9)'),
 (6, 'D(2)'),
 (6, 'Bbmaj7sus4/Eb'),
 (6, 'B11b9'),
 (6, 'G(maj7)'),
 (6, 'G7+5-9'),
 (5, 'Em+'),
 (5, 'G-'),
 (5, 'C7#5#9'),
 (5, 'Bb7#5#9'),
 (5, 'C#9(#5)'),
 (5, 'E*'),
 (5, 'Ammaj9'),
 (5, 'D#(b5)'),
 (5, 'Bb-'),
 (4, 'Cm#'),
 (4, 'Bb6a'),
 (4, 'Fm7#5'),
 (4, 'Gm7b5b9'),
 (4, 'Gmaj7sus4/C'),
 (4, 'FmMaj7'),
 (4, 'F9/11'),
 (4, 'F(5)'),
 (4, 'F#dim(F)'),
 (4, 'B7#5#9'),
 (4, 'D(b5)'),
 (4, 'Em7(VII)'),
 (4, 'G7s4'),
 (4, 'Bbmaj7+5'),
 (4, 'B-'),
 (4, 'Eaddb13'),
 (3, 'E7*'),
 (3, 'Bmadd4'),
 (3, 'Cmin6'),
 (3, 'Emaj7b13/Ab'),
 (3, 'Bm*'),
 (3, 'LA'),
 (3, 'G#madd6'),
 (3, 'F#m7(b5)'),
 (3, 'Bbmaj9+11'),
 (3, 'Am*'),
 (3, 'Cm7/9

In [63]:
def any_not_found(chords):
    return len([chord for chord in not_found if chord[1] in chords]) > 0

In [64]:
any_not_found(['A'])

False

In [65]:
any_not_found(['Go'])

False

In [66]:
len(df[df['chords'].apply(lambda x: any_not_found(x))]['url'])

108

'https://tabs.ultimate-guitar.com/tab/fugees/killing-me-softly-chords-50470'

In [68]:
Chord('A7/G').components()

['G', 'A', 'C#', 'E', 'G']

In [27]:
from pychord_jl.constants import QUALITY_DICT

In [37]:
[list(item[1]) for item in QUALITY_DICT.items()]

[[0, 7],
 [0, 7],
 [0, 7],
 [0, 4, 7],
 [0, 4, 7],
 [0, 4, 7],
 [0, 3, 7],
 [0, 3, 7],
 [0, 3, 7],
 [0, 3, 6],
 [0, 3, 6],
 [0, 4, 8],
 [0, 4, 8],
 [0, 4, 8],
 [0, 4, 8],
 [0, 2, 7],
 [0, 2, 7],
 [0, 5, 7],
 [0, 5, 7],
 [0, 5, 7],
 [0, 4, 7, 9],
 [0, 4, 7, 9],
 [0, 4, 7, 10],
 [0, 4, 7, 10],
 [0, 4, 7, 10],
 [0, 4, 6, 10],
 [0, 4, 6, 10],
 [0, 4, 6, 10],
 [0, 4, 6, 11],
 [0, 4, 8, 10],
 [0, 4, 8, 10],
 [0, 4, 8, 10],
 [0, 4, 8, 10],
 [0, 2, 7, 9],
 [0, 2, 7, 10],
 [0, 2, 7, 10],
 [0, 5, 7, 10],
 [0, 5, 7, 10],
 [0, 5, 7, 10],
 [0, 5, 7, 10],
 [0, 3, 7, 9],
 [0, 3, 7, 10],
 [0, 3, 7, 10],
 [0, 3, 7, 10],
 [0, 3, 7, 10],
 [0, 3, 7, 10],
 [0, 3, 7, 10],
 [0, 3, 6, 10],
 [0, 3, 6, 10],
 [0, 3, 6, 9],
 [0, 4, 7, 11],
 [0, 4, 7, 11],
 [0, 4, 7, 11],
 [0, 4, 7, 11],
 [0, 4, 7, 11],
 [0, 4, 8, 11],
 [0, 3, 7, 11],
 [0, 3, 7, 11],
 [0, 3, 7, 11],
 [0, 4, 7, 14],
 [0, 4, 7, 14],
 [0, 3, 7, 14],
 [0, 4, 7, 14],
 [0, 2, 4, 7],
 [0, 4, 5, 7],
 [0, 4, 7, 17],
 [0, 2, 4, 7],
 [0, 4, 5, 7],
 [0, 4, 7,

In [38]:
functools.reduce(lambda list1,list2: list(set([*list1, *list2])), [list(item[1]) for item in QUALITY_DICT.items()])

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 17, 18, 20, 21]

In [30]:
QUALITY_DICT.

odict_items([('5', (0, 7)), ('-5', (0, 7)), ('sus', (0, 7)), ('', (0, 4, 7)), ('maj', (0, 4, 7)), ('M', (0, 4, 7)), ('m', (0, 3, 7)), ('min', (0, 3, 7)), ('mi', (0, 3, 7)), ('dim', (0, 3, 6)), ('o', (0, 3, 6)), ('aug', (0, 4, 8)), ('(aug)', (0, 4, 8)), ('+5', (0, 4, 8)), ('+', (0, 4, 8)), ('sus2', (0, 2, 7)), ('(sus2)', (0, 2, 7)), ('sus4', (0, 5, 7)), ('(sus4)', (0, 5, 7)), ('s4', (0, 5, 7)), ('6', (0, 4, 7, 9)), ('add6', (0, 4, 7, 9)), ('7', (0, 4, 7, 10)), ('(7)', (0, 4, 7, 10)), ('(M7)', (0, 4, 7, 10)), ('7-5', (0, 4, 6, 10)), ('7_5', (0, 4, 6, 10)), ('7b5', (0, 4, 6, 10)), ('maj7b5', (0, 4, 6, 11)), ('7+5', (0, 4, 8, 10)), ('7+', (0, 4, 8, 10)), ('7#5', (0, 4, 8, 10)), ('7(#5)', (0, 4, 8, 10)), ('6sus2', (0, 2, 7, 9)), ('7sus2', (0, 2, 7, 10)), ('maj7sus2', (0, 2, 7, 10)), ('7sus4', (0, 5, 7, 10)), ('7(sus)', (0, 5, 7, 10)), ('sus7', (0, 5, 7, 10)), ('7sus', (0, 5, 7, 10)), ('m6', (0, 3, 7, 9)), ('m7', (0, 3, 7, 10)), ('m(7)', (0, 3, 7, 10)), ('7m', (0, 3, 7, 10)), ('min7', (0, 3,